# Amadeus Data Science Challenge

## Ejercicio 1. Cuenta el número de línease del documento "bookings.csv" 
- Posibles formas de solucionarlo:
    - Usando la línea de comandos
    - Usando bucles en Python
    - Usando Pandas

### Usando la línea de comandos

In [1]:
! pwd

/home/dsc/git_shared/master_data_science_kschool/07_amadeus_data_science_challenge


In [2]:
cd /home/dsc/Data/challenge

/home/dsc/Data/challenge


In [3]:
ls -l

total 1013840
-rw-rw-r-- 1 dsc dsc 554970628 abr  2  2018 bookings.csv.bz2
-rw-rw-r-- 1 dsc dsc 483188920 abr  2  2018 searches.csv.bz2


- Vemos que es muy pesado (+500MB), por lo que es mejor trabajar con él comprimido (en este caso, en formato bz2)
    - Se podría descomprimir con `! bunzip2` y comprimir con `! bzip2`, pero mejor trabajar con este formato bzip
    - El formato 'bzip' es aconsejable para archivos de +128MB
    - Es más recomendable que '.zip', pero más lento. 
    - Con '.bzip2' nos aseguramos que cada archivo comprimido solo tenga un archivo, algo que no pasa con los '.zip'

In [4]:
# El archivo es muy pesado, durará un rato
! bzcat bookings.csv.bz2 | wc -l

10000011


- Mediante línea de comandos el ejercicio ya estaría resuelto: el archivo tiene 10.000.011 líneas.

In [8]:
# Para ejercicios posteriores se va a exportar una muestra de las 10.000 primeras líneas
! bzcat ./bookings.csv.bz2 | head -10000 > ./bookings_sample.csv
# Si en vez de una muestra de las 10.000 primeras líneas quisiera una mezcla aleatoria, usaría 'sample'
! bzcat ./bookings.csv.bz2 | sample -10000 > ./bookings_sample_2.csv
# La misma muestra se comprimirá en 'bzip2' para trabajar en el mismo formato que la entrada
! bzip2 bookings_sample.csv


bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = ./bookings.csv.bz2, output file = (stdout)


### Usando Python
- Se puede calcular el número de líneas usando bucles
- Para evitar que se tarde mucho en realizar cálculos, se trabajará sobre las 10.000 primeras líneas extraídas

In [12]:
import bz2

fileBz2 =  bz2.BZ2File("./bookings_sample.csv.bz2")
type(fileBz2)

bz2.BZ2File

In [13]:
#Para evitar retardos en el cálculo se utilizará la muestra de 10.000 líneas, pero el concepto es el mismo.
k = 0

for line in fileBz2:
    k += 1
    
print(k)

10000


In [14]:
# Mucho ojo con volver a correr el bucle ahora, ya que dará cero al apuntar ahora el puntero al final.
k = 0

for line in fileBz2:
    k += 1

print(k)

0


### Mejorando la solución de Python
- ¿El fichero está cerrado?

In [21]:
fileBz2.closed

False

- Se cierra el fichero

In [22]:
fileBz2.close()

In [23]:
fileBz2.closed

True

- Pero podría haberse usado un `with` para hacerlo más sencillo.

In [16]:
import bz2
with bz2.BZ2File("./bookings_sample.csv.bz2") as fileBz2:
    k = 0
    for line in fileBz2:
        k += 1
    print(k)
print(fileBz2.closed)

10000
True


### Mejorando aún más la solución de Python.
- Se pueden manejar excepciones con un `try/except`

In [18]:
import bz2
try:
    with bz2.BZ2File("./bookings_sample.csv.bz2") as fileBz2:
        k = 0
        for line in fileBz2:
            k += 1
except:
    print("Error counting lines")
        
print(k)

10000


- Podemos dar un paso mas teniendo en cuenta más posibles errores:

In [17]:
import bz2
try:
    with bz2.BZ2File("./bookings_sample.csv.bz2") as fileBz2:
        k = 0
        for line in fileBz2:
            k += 1
except FileNotFoundError:
    print("File Not Found")
except:
    print("Unexpected error")
        
print(k)

10000


- Una persona más experimentada hubiera hecho que salte por defecto el error del mensaje

In [28]:
import bz2
try:
    with bz2.BZ2File("./bookisdfngs_sample.csv.bz2") as fileBz2:
        k = 0
        for line in fileBz2:
            k += 1
except FileNotFoundError as message:
    print(message)
except:
    print("Unexpected error")
        
print(k)

[Errno 2] No such file or directory: './bookisdfngs_sample.csv.bz2'
10000


- O incluso poniendo un `enumerate`

In [19]:
import bz2
try:
    with bz2.BZ2File("./bookings_sample.csv.bz2") as fileBz2:
        for k, line in enumerate(fileBz2):
            pass
        print(k+1)
except FileNotFoundError as message:
    print(message)
except ValueError:
    print("value error")
except:
    print("unexpected error")

10000


# Solución 3. Mediante el uso de Pandas

In [22]:
# Puedo ver el nombre de las columnas mediante un 'head -1' para seleccionar solo el header y realizar una transposición
! bzcat bookings_sample.csv.bz2 | head -1 |tr ^ "\n"


bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = bookings_sample.csv.bz2, output file = (stdout)
act_date           
source
pos_ctry
pos_iata
pos_oid  
rloc          
cre_date           
duration
distance
dep_port
dep_city
dep_ctry
arr_port
arr_city
arr_ctry
lst_port
lst_city
lst_ctry
brd_port
brd_city
brd_ctry
off_port
off_city
off_ctry
mkt_port
mkt_city
mkt_ctry
intl
route          
carrier
bkg_class
cab_class
brd_time           
off_time           
pax
year
month
oid      


- Puedo importar el dataframe en Pandas y ver las columnas de la siguiente manera:
    - **Ojo**: se puede ver que el nombre de algunas columnas tiene espacios indeseados que debo saber manejar. 

In [21]:
import pandas as pd
df = pd.read_csv("bookings_sample.csv.bz2", sep="^")
df.columns

Index(['act_date           ', 'source', 'pos_ctry', 'pos_iata', 'pos_oid  ',
       'rloc          ', 'cre_date           ', 'duration', 'distance',
       'dep_port', 'dep_city', 'dep_ctry', 'arr_port', 'arr_city', 'arr_ctry',
       'lst_port', 'lst_city', 'lst_ctry', 'brd_port', 'brd_city', 'brd_ctry',
       'off_port', 'off_city', 'off_ctry', 'mkt_port', 'mkt_city', 'mkt_ctry',
       'intl', 'route          ', 'carrier', 'bkg_class', 'cab_class',
       'brd_time           ', 'off_time           ', 'pax', 'year', 'month',
       'oid      '],
      dtype='object')

In [26]:
#Puedo seleccionar solo una columna para ver el largo y poder contar las filas:
df = pd.read_csv("bookings_sample.csv.bz2", sep="^", usecols=["pax"])

#Con 'shape' obtengo una tupla con su largo y su ancho.
df.shape

#Me da 9.999 porque falta una fila que ha usado como header.

(9999, 1)

In [25]:
#Al ser una tupla puedo hacer slicing para obtener solo el largo
df.shape[0]

9999